# Aggregate label data 

Groups together all the manually labelled data

In [10]:
%%capture
pip install -r ../../requirements.txt

In [51]:
import json 

theme_colors_path = "../theme_colors.json"
with open(theme_colors_path, "r") as f:
    color_palette = json.load(f)
print(json.dumps(color_palette, indent=4))


{
    "color_palette": {
        "primary": "#1E90FF",
        "secondary": "#FF6347",
        "accent": "#32CD32",
        "background": "#F5F5F5",
        "text": "#333333",
        "warning": "#FFA500",
        "success": "#28A745",
        "error": "#DC3545"
    }
}


In [23]:
import pandas as pd 
import numpy as np 
import os 


In [18]:
empty_df = pd.DataFrame() 
empty_df.empty

True

In [45]:
# REPLACE: Go through the google drive folder and combine the csvs 

# Iterate through all the files in a folder location 
folder_path_location = 'labeled_data'
final_df = pd.DataFrame()

for file in os.listdir(folder_path_location):
    file_path = os.path.join(folder_path_location, file)
    if "combined" in file_path: 
        continue
    file_df = pd.read_csv(file_path)
    print(file_path, file_df.shape[0])
    if final_df.empty : 
        final_df = file_df 
    else: 
        final_df  = pd.concat([final_df, file_df], ignore_index=True)

print("Final file size: ",  final_df.shape[0])

# Save the combined DataFrame to a new CSV file
output_file = os.path.join(folder_path_location, 'combined_output.csv')
final_df.to_csv(output_file)
print(output_file)


labeled_data/aserban_labeled_data_2.csv 20
labeled_data/aserban_labeled_data_3.csv 5
labeled_data/aserban_labeled_data_1.csv 20
Final file size:  45
labeled_data/combined_output.csv


In [43]:
import sys
sys.path.append("../../scripts/")

from data_collection import authenticate_google_drive, grab_google_drive_folder_data

drive = authenticate_google_drive('../0_data_collection/credentials/google_drive_client_secret.json')

In [44]:
## Clean up
file_path = "labeled_data_3.29.csv"
combined_labeled_data  = grab_google_drive_folder_data(drive=drive,credential_file="../0_data_collection/credentials/google_drive_folder_id.json",filename=file_path)
combined_labeled_data.head()

# Make sure there are no duplicates in labeling 
combined_labeled_data_grouped = pd.DataFrame(combined_labeled_data.groupby(['submission_id','subredit_topic','search_query', 'combine_text', 'url']).agg({'label': ['count', 'min', 'max']})).reset_index()
combined_labeled_data_grouped.columns = ['_'.join(col) for col in combined_labeled_data_grouped.columns]
combined_labeled_data_grouped.columns

Successfully loaded 'labeled_data_3.29.csv' into a DataFrame!


Index(['submission_id_', 'subredit_topic_', 'search_query_', 'combine_text_',
       'url_', 'label_count', 'label_min', 'label_max'],
      dtype='object')

In [45]:
# Check if there are instances that the labels are not the same 
combined_labeled_data_grouped[combined_labeled_data_grouped['label_count'] > 1]
combined_labeled_data_grouped['same_value'] = combined_labeled_data_grouped['label_min'] == combined_labeled_data_grouped['label_max']
print("Number of conflicting labels:  ", combined_labeled_data_grouped[~combined_labeled_data_grouped['same_value']].shape[0])

# Drop instances where the label was conflicting
conflicting_submission_ids = list(combined_labeled_data_grouped[~combined_labeled_data_grouped['same_value']]['submission_id_'])
combined_labeled_data = combined_labeled_data[~combined_labeled_data['submission_id'].isin(conflicting_submission_ids)].reset_index(drop=True)


Number of conflicting labels:   1


In [47]:
# Conflicting Label Investiagtionn Section 
# submission_id = "1iq9vh3"
# combined_labeled_data[combined_labeled_data['submission_id'] == submission_id]['combine_text'][114]

In [58]:
# See the distribution of the labels 
print(combined_labeled_data['label'].value_counts() )
print('\n')
print(combined_labeled_data['label'].value_counts(normalize=True))

Neutral     101
Negative     26
Positive      3
Name: label, dtype: int64


Neutral     0.776923
Negative    0.200000
Positive    0.023077
Name: label, dtype: float64


In [59]:
# Delete Later

combined_labeled_data.to_csv("labeled_data/combined_output_new.csv")
